In [ ]:
!pip install -qq torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -qq mmcv-full==1.6.0 -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html
!git clone https://github.com/open-mmlab/mmdetection
%cd mmdetection

!pip install -e .

In [ ]:
!pip install -q --upgrade wandb

# Imports

In [ ]:
import wandb
import os
import shutil
import sys
import json
import torch,torchvision
from collections import defaultdict
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
from mmcv import Config

import mmdet
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.apis import init_detector, inference_detector

from tqdm import tqdm
import random
import numpy as np
from pathlib import Path

In [ ]:
seed = 42
set_random_seed(seed, deterministic=False)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
wandb.login(key="")

# Constants

In [ ]:
FOLD = 0
EPOCHS = 12

In [ ]:
PROJECT_NAME = 'Final-Covid19-Detection'
NOTEBOOK_NAME = ''
EXP_NAME = f'RetinaNet_Fold_{FOLD}'


In [ ]:
train_ann_file = f'/working/coco_train_512x512_fold_{FOLD}.json'
val_ann_file = f'/working/coco_val_512x512_fold_{FOLD}.json'

# Model Training

In [ ]:
model_name = 'retinanet/retinanet_x101_64x4d_fpn_1x_coco'

In [ ]:
baseline_cfg_path = f'/working/mmdetection/configs/{model_name}.py'
cfg = Config.fromfile(baseline_cfg_path)
job_folder = f'/working/retinanet'
if not os.path.exists(job_folder):
    os.makedirs(job_folder)
cfg.work_dir = job_folder


In [ ]:
cfg.model=dict(
    type='RetinaNet',
    backbone=dict(
        type='ResNeXt',
        depth=101,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=None,
        groups=64,
        base_width=4),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        start_level=1,
        add_extra_convs='on_input',
        num_outs=5),
    bbox_head=dict(
        type='RetinaHead',
        num_classes=1,
        in_channels=256,
        stacked_convs=4,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            octave_base_scale=4,
            scales_per_octave=3,
            ratios=[0.5, 1.0, 2.0],
            strides=[8, 16, 32, 64, 128],),
            #strides=[4, 8, 16, 32]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='FocalLoss',
            use_sigmoid=True,
            gamma=2.0,
            alpha=0.25,
            loss_weight=1.0),
        loss_bbox=dict(type='L1Loss', loss_weight=1.0)),
    train_cfg=dict(
        assigner=dict(
            type='MaxIoUAssigner',
            pos_iou_thr=0.5,
            neg_iou_thr=0.4,
            min_pos_iou=0,
            ignore_iof_thr=-1),
        allowed_border=-1,
        pos_weight=-1,
        debug=False),
    test_cfg=dict(
        nms_pre=1000,
        min_bbox_size=0,
        score_thr=0.05,
        nms=dict(type='nms', iou_threshold=0.5),
        max_per_img=100))

In [ ]:
albu_train_transforms = [
    #dict(type='RandomResizedCrop',height=image_scale[0],width=image_scale[1],scale=(0.25,1.0),ratio=(0.75,1.3333),interpolation=1,p=1.0),
    #dict(type='Equalize',mode='cv',p=1,always_apply=True),
    dict(type='Rotate',limit=10,interpolation=1,border_mode=0,value=0,p=0.25),
    dict(type='HorizontalFlip',p=0.5),
    dict(
        type="OneOf",
        transforms=[
            dict(type="MotionBlur", p=0.2),
            dict(type="MedianBlur", p=0.1, blur_limit=3),
            dict(type='Blur',p=0.1,blur_limit=3)
        ],
        p=0.25),
    dict(type='ShiftScaleRotate', shift_limit=0.0625,scale_limit=0.15, rotate_limit=15, p=0.4),
    dict(type='RandomBrightnessContrast', brightness_limit=0.2,
         contrast_limit=0.2, p=0.1)
]


In [ ]:
cfg.train_pipeline = train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='EqualizeTransform',prob=1),
    dict(
        type='Albu',
        transforms=albu_train_transforms,
        keymap=dict(img='image', gt_bboxes='bboxes'),
        update_pad_shape=False,
        skip_img_without_anno=True,
        bbox_params=dict(
            type='BboxParams',
            format='pascal_voc',
            label_fields=['gt_labels'],
            filter_lost_elements=True,
            min_visibility=0.1,
            min_area=1)),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    #dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=1),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]

In [ ]:
cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='EqualizeTransform',prob=1),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='Equalize',mode='cv',p=1,always_apply=True),
            dict(type='RandomFlip'),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img'])
        ])
]

In [ ]:
cfg.data = dict(
    samples_per_gpu=8,
    workers_per_gpu=2,
    train=dict(
        type='CocoDataset',
        ann_file=train_ann_file,
        img_prefix='/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512',
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(type='LoadAnnotations', with_bbox=True),
            dict(type='EqualizeTransform',prob=1),
            dict(
                type='Albu',
                transforms=[dict(type='CLAHE',p=1,always_apply=True),
                            *albu_train_transforms],
                keymap=dict(img='image', gt_bboxes='bboxes'),
                update_pad_shape=False,
                skip_img_without_anno=True,
                bbox_params=dict(
                    type='BboxParams',
                    format='pascal_voc',
                    label_fields=['gt_labels'],
                    filter_lost_elements=True,
                    min_visibility=0.1,
                    min_area=1)),
            dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
            dict(type='RandomFlip', flip_ratio=0.5),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='Pad', size_divisor=1),
            dict(type='DefaultFormatBundle'),
            dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
        ],
        classes=('Covid_Opacity', ),
        filter_empty_gt=True),
    val=dict(
        type='CocoDataset',
        ann_file=val_ann_file,
        img_prefix='/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512',
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(type='EqualizeTransform',prob=1),
            dict(
                type='MultiScaleFlipAug',
                img_scale=(1333, 800),
                flip=False,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='ImageToTensor', keys=['img']),
                    dict(type='Collect', keys=['img'])
                ])
        ],
        classes=('Covid_Opacity', ),
        filter_empty_gt=True),
    test=dict(
        type='CocoDataset',
        ann_file=val_ann_file,
        img_prefix='data/images/train/',
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(type='EqualizeTransform',prob=1),
            dict(
                type='MultiScaleFlipAug',
                img_scale=(1333, 800),
                flip=False,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='ImageToTensor', keys=['img']),
                    dict(type='Collect', keys=['img'])
                ])
        ],
        classes=('Covid_Opacity', ),
        filter_empty_gt=True))

In [ ]:
cfg.evaluation = dict(interval=1, metric='bbox', save_best='bbox_mAP_50')
cfg.optimizer = dict(type='SGD', lr=0.00003, momentum=0.9, weight_decay=0.0001)
cfg.optimizer_config = dict(grad_clip=None)
cfg.lr_config = dict(
    policy='CosineAnnealing',
    by_epoch=False,
    warmup='linear',
    warmup_iters=500,
    warmup_ratio=0.001,
    min_lr=0.0)
cfg.runner = dict(type='EpochBasedRunner', max_epochs=EPOCHS)
cfg.checkpoint_config = dict(interval=5)

cfg.custom_hooks = [dict(type='NumClassCheckHook')]
cfg.dist_params = dict(backend='nccl')
cfg.log_level = 'INFO'
cfg.load_from = 'https://download.openmmlab.com/mmdetection/v2.0/retinanet/retinanet_x101_64x4d_fpn_1x_coco/retinanet_x101_64x4d_fpn_1x_coco_20200130-366f5af1.pth'
cfg.resume_from = None
cfg.workflow = [('train', 1)]
cfg.classes = ('Covid_Opacity', )

cfg.fp16 = dict(loss_scale=512.0)
cfg.total_epochs = EPOCHS
cfg.work_dir = '/kaggle/working/retinanet'
cfg.gpu_ids = 0

In [ ]:
cfg.log_config = dict(
    interval=10,
    hooks=[dict(type='TextLoggerHook'),
        dict(type='WandbLoggerHook',
         init_kwargs=dict(project=PROJECT_NAME,
                          name=f'{EXP_NAME}_{dt_string}',
                          job_type='model_train',
                          entity='alvaromoureupm',
                         notes=NOTEBOOK_NAME,
                         config=cfg._cfg_dict.to_dict()),
         log_artifact=True,
         interval = 100
                            )])

In [ ]:
cfg.dump('/working/retinanet/retinanet.py')

In [ ]:
!python tools/train.py /kaggle/working/retinanet/retinanet.py